In [ ]:
import sys
!{sys.executable} -m pip --version 

In [ ]:
!{sys.executable} -m pip install -q transformers torch scikit-learn sentencepiece safetensors

In [ ]:
import os
os.environ["TRANSFORMERS_NO_TORCHVISION_IMPORTS"] = "1"
import os, torch, torch.nn.functional as F, pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report, confusion_matrix

os.environ["CUDA_VISIBLE_DEVICES"] = ""
DEVICE   = "cpu"                             
MODEL_ID = "MayZhou/e5-small-lora-ai-generated-detector"

In [ ]:
tok   = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_ID).to(DEVICE)
model.eval()

In [ ]:
import torch.nn.functional as F
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

df = pd.read_csv("test_10k.csv")          # 'data' + 'label'
assert {"data", "label"} <= set(df.columns)
print(df.head())

In [ ]:
from tqdm.auto import tqdm

def predict_probs(texts, batch=32, max_len=256):
    probs = []
    for i in tqdm(range(0, len(texts), batch), desc="Batches"):
        enc = tok(
            texts[i:i+batch].tolist(),
            padding=True, truncation=True, max_length=max_len,
            return_tensors="pt"
        ).to(model.device)

        with torch.no_grad():
            logits = model(**enc).logits
            p = torch.softmax(logits, dim=-1)[:, 1]
        probs.extend(p.cpu().numpy())
    return probs

In [ ]:
df["prob_ai"] = predict_probs(df["data"])
df["pred"]    = (df["prob_ai"] >= 0.5).astype(int)

In [ ]:
df.to_csv("test_10k_with_preds.csv", index=False)